In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from sklearn.cluster import DBSCAN
import folium

# Conversão de dados tabulares em dados vetoriais
def load_ocorrencias(path):
    df = pd.read_csv(path).dropna(subset=["latitude", "longitude"])
    gdf = gpd.GeoDataFrame(
        df,
        geometry=[Point(xy) for xy in zip(df.longitude, df.latitude)],
        crs="EPSG:4326")
    # Garantia explícita do sistema de referência
    assert gdf.crs.to_string() == "EPSG:4326"
    return gdf

# Reprojeção cartográfica e clusterização espacial (DBSCAN)
def clusterizar(gdf, eps=200, min_samples=5):
    # Projeção métrica (metros) para cálculo correto de distância
    gdf_proj = gdf.to_crs(epsg=3857)
    coords = list(zip(gdf_proj.geometry.x, gdf_proj.geometry.y))
    gdf["cluster"] = DBSCAN(
        eps=eps,
        min_samples=min_samples
    ).fit_predict(coords)
    return gdf

# Geração de mapa interativo com destaque para hotspots
def gerar_mapa(gdf, output):
    m = folium.Map(
        location=[gdf.geometry.y.mean(), gdf.geometry.x.mean()],
        zoom_start=13,
        tiles="CartoDB positron")
    for _, row in gdf.iterrows():
        is_hotspot = row.cluster != -1

        folium.CircleMarker(
            location=[row.geometry.y, row.geometry.x],
            radius=6 if is_hotspot else 3,
            color="red" if is_hotspot else "gray",
            fill=True,
            fill_opacity=0.7 if is_hotspot else 0.3,
            popup=(
                f"Tipo: {row.get('tipo', 'N/A')}<br>"
                f"Cluster: {row.cluster}")).add_to(m)
    m.save(output)

# Pipeline de execução
if __name__ == "__main__":
    gdf = load_ocorrencias("data/raw/ocorrencias.csv")
    gdf = clusterizar(gdf)
    # Persistência do resultado processado
    gdf.to_file("data/processed/ocorrencias_clusterizadas.gpkg", driver="GPKG")
    gerar_mapa(gdf, "outputs/maps/hotspots.html")